# Preprocess Covid-19 dataset

FakeCovid is the first multilingual cross-domain dataset of 7623 fact-checked news articles for COVID-19, collected from 04/01/2020 to 01/07/2020. For more information, check out this link : 
https://github.com/Gautamshahi/FakeCovid

## Importing packages 

In [1]:
import pandas as pd
import os.path as op
from sklearn.preprocessing import LabelEncoder
import numpy as np
import seaborn as sns
import glob
import matplotlib.pyplot as plt

In [2]:
DATA_DIR = '../../Data'

## Training Data :

### Putting the three dataset (train, valid and test) in one dataset 

In [3]:
path = DATA_DIR+'/Covid-19/Unprocessed_data' # use your path
all_files = glob.glob(path + "/*.csv")
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [4]:
train=frame.copy()

In [5]:
train

ID                                 ref_category_title  \
0      FC1  FALSE: The coronavirus is an amplified bacteri...   
1      FC2  FALSE: A law allows people to go for a run dur...   
2      FC3  False: Chinese converting to Islam after reali...   
3      FC4  False: Bat market and bat meat are being sold ...   
4      FC5  False: You can self-diagnose COVID-19 by holdi...   
...    ...                                                ...   
12800  NaN                                                NaN   
12801  NaN                                                NaN   
12802  NaN                                                NaN   
12803  NaN                                                NaN   
12804  NaN                                                NaN   

                                                 ref_url  \
0      https://www.poynter.org/?ifcn_misinformation=t...   
1      https://www.poynter.org/?ifcn_misinformation=a...   
2      https://www.poynter.org/?ifcn_misinformation=c...   
3      https://www.poynter.org/?ifcn_misinformation=b...   
4      https://www.poynter.org/?ifcn_misinformation=y...   
...                                                  ...   
12800  https://www.poynter.org/?ifcn_misinformation=c...   
12801                                                NaN   
12802  https://www.poynter.org/?ifcn_misinformation=t...   
12803  https://www.poynter.org/?ifcn_misinformation=t...   
12804  https://www.poynter.org/?ifcn_misinformation=g...   

                                                  pageid  \
0      https://www.poynter.org/ifcn-covid-19-misinfor...   
1      https://www.poynter.org/ifcn-covid-19-misinfor...   
2      https://www.poynter.org/ifcn-covid-19-misinfor...   
3      https://www.poynter.org/ifcn-covid-19-misinfor...   
4      https://www.poynter.org/ifcn-covid-19-misinfor...   
...                                                  ...   
12800                                                NaN   
12801                                                NaN   
12802                                                NaN   
12803                                                NaN   
12804                                                NaN   

                 verifiedby                     country       class  \
0            La Silla Vacía                    Colombia       FALSE   
1                Newtral.es                       Spain       FALSE   
2             FactCrescendo                       India       False   
3       France 24 Observers                      France       False   
4              Agência Lupa                      Brazil       False   
...                     ...                         ...         ...   
12800      Bolivia Verifica              Bolivia, Spain       FALSE   
12801         Efecto Cocuyo                   Venezuela       FALSE   
12802             La Nación                  Costa Rica       FALSE   
12803                Misbar   Middle East, North Africa  Misleading   
12804           LeadStories                         USA       FALSE   

                                                   title published_date  \
0       The coronavirus is an amplified bacteria rela...    2020/06/17    
1       A law allows people to go for a run during th...    2020/04/09    
2       Chinese converting to Islam after realizing t...    2020/02/20    
3       Bat market and bat meat are being sold in Wuhan.    2020/01/27    
4       You can self-diagnose COVID-19 by holding you...    2020/03/16    
...                                                  ...            ...   
12800                                                NaN     2020-04-08   
12801                                                NaN     2020-02-03   
12802                                                NaN     2020-04-06   
12803                                                NaN     2020-02-12   
12804                                                NaN     2020-03-11   

            country1  ... count

## Getting rid of the columns that we don't need 

In [6]:
train=train.drop(columns=['ID','ref_category_title','ref_url','pageid','verifiedby','country','published_date','country1','country2','country3','country4','ref_source','category','FCID','ref_title','SM_Link'])

## Selectting only tweets that are written in english 

In [7]:
train['lang'].value_counts()

en            2845
English       2116
es            1321
Spanish        879
tl             532
              ... 
fi               1
Vietnamese       1
sk               1
Finnish          1
vi               1
Name: lang, Length: 80, dtype: int64

In [8]:
train=train[(train['lang']=='English') | (train['lang']=='en')]

In [9]:
train['lang'].value_counts()

en         2845
English    2116
Name: lang, dtype: int64

## We will drop the column of 'lang' because we don't need it anymore 

In [10]:
train=train.drop(columns=['lang'])

## Rename the columns 

In [11]:
train = train.rename(columns={'class': 'label',  'content_text':'text','article_source':'author'})

## Deal with null values 

In [12]:
train.isnull().sum()

label              0
title           2116
author             0
source_title      51
text               0
dtype: int64

In [13]:
train =train.reset_index(drop=True)

   ##  We have kept the 'source_title' column so that we can replace the null values in 'title' column by  'source_title'  values

In [14]:
mask = train.isna()
train=train[~((mask.source_title == True) & (mask.title == True))] 

In [17]:
train

label                                              title  \
0           False   Chinese converting to Islam after realizing t...   
1      MISLEADING   Captions on a reuploaded video about the U.S....   
2     Mostly True   Ghana has 307 ambulances with mobile ventilat...   
3           FALSE   “Governor Andy Beshear has authorized Kentuck...   
4           False   Photo shows food being distributed to Rwandan...   
...           ...                                                ...   
4941        FALSE                                                NaN   
4942        FALSE                                                NaN   
4943        FALSE                                                NaN   
4944        FALSE                                                NaN   
4945        FALSE                                                NaN   

                                                 author  \
0     https://english.factcrescendo.com/2020/02/20/c...   
1     https://verafiles.org/articles/vera-files-fact...   
2     https://ghanafact.com/fact-check-does-ghanas-3...   
3     https://www.politifact.com/factchecks/2020/may...   
4     https://africacheck.org/fbcheck/food-distribut...   
...                                                 ...   
4941  https://hindi.boomlive.in/fake-news/video-of-a...   
4942  https://www.vishvasnews.com/english/health/fac...   
4943  https://www.snopes.com/fact-check/resident-evi...   
4944  https://www.boomlive.in/fake-news/images-of-ba...   
4945  https://hoax-alert.leadstories.com/3471891-fac...   

                                           source_title  \
0     Are Chinese people converting to Islam in fear...   
1     VERA FILES FACT CHECK: Remdesivir to ‘end’ COV...   
2     Fact-check: Does Ghana have 307 ambulances wit...   
3     PolitiFact | No, Kentucky teachers won’t be co...   
4     Food distribution during Rwanda’s coronavirus ...   
...                                                 ...   
4941  Canine coronavirus vaccine is for dogs – not r...   
4942  Fact Check: Post claiming alcohol kills corona...   
4943  Is the ‘Umbrella Corporation’ Logo Oddly Simil...   
4944  Images Of Baby With Birth Defect Falsely Linke...   
4945  Fact Check: Memes With Advice For Preventing C...   

                                                   text  
0     The fact behind every news!, Ever since the Wo...  
1     AUTHOR, VERA Files, DATE, May 08, 2020, SHARE,...  
2     Source: Dr Anthony Nsiah Asare, Verdict: Mostl...  
3     More Info, Trying to focus on school work at h...  
4     A photo of hundreds of neat piles of bedding a...  
...                                                 ...  
4941  A photo shared on Facebook in South Africa sho...  
4942  A viral post on social media claims that alcoh...  
4943  , The claim that “Corona” is an anagram for “R...  
4944  BOOM found that the baby was born at a village...  
4945  The most widespread meme is this one, claiming...  

[4946 rows x 5 columns]

In [18]:
train =train.reset_index(drop=True)

In [19]:
# replace na in title
for n in range (0,len(train)):
    if pd.isnull(train['title'][n]):
        train['title'][n]= str(train['title'][n]).replace('nan', str(train['source_title'][n]))
train.isnull().sum()

label            0
title            0
author           0
source_title    36
text             0
dtype: int64

In [20]:
train =train.reset_index(drop=True)

## Now, we don't need the 'source_title' column

In [21]:
train=train.drop(columns=['source_title'])

In [22]:
train.isnull().sum()

label     0
title     0
author    0
text      0
dtype: int64

## Deal with the labels 

In [23]:
train['label']=train['label'].str.lower()

In [24]:
train['label'].value_counts()

false                          3866
misleading                      403
mostly false                    196
mixture                          62
true                             50
news                             40
partly false                     32
pants on fire!                   31
half true                        29
mostly true                      26
pants on fire                    21
unproven                         20
correct attribution              18
miscaptioned                     16
fake                             13
no evidence                      13
(org. doesn't apply rating)      13
misattributed                    11
inaccurate                        9
partially false                   8
labeled satire                    7
incorrect                         6
scam                              6
two pinocchios                    5
false connection                  4
partly true                       4
not true                          4
unsupported                 

In [25]:
train["label"].replace({"true but": "0", "fake news": "1","pants on fire!":"1","false":"1","misleading":"1","mostly false":"1","partly false":"1", "true":"0","mostly true":"0","pants on fire":"1","fake":"1","inaccurate":"1","labeled satire ":"1","scam":"1","misplaced context":"1","false connection":"1","unlikely":"1","false headline":"1","mostly inaccurate":"1","conspiracy theory":"1","mosly false":"1","not true":"1","incorrect":"1","labeled satire":"1","half true":"1","partially false":"1" }, inplace=True)
train['label'].value_counts()

1                              4644
0                                76
mixture                          62
news                             40
unproven                         20
correct attribution              18
miscaptioned                     16
no evidence                      13
(org. doesn't apply rating)      13
misattributed                    11
two pinocchios                    5
partly true                       4
four pinocchios                   4
unsupported                       4
three pinocchios                  3
partially correct                 2
mixed                             2
in dispute                        2
partially true                    2
old policy                        1
context                           1
collections                       1
insufficient evidence             1
out of context                    1
Name: label, dtype: int64

In [26]:
train=train[train[['label']].apply(lambda x: x[0].isdigit(), axis=1)]

In [27]:
train['label'].value_counts()

1    4644
0      76
Name: label, dtype: int64

## Transforming the label column into integer 

In [28]:
train['label'] =train['label'].astype(int)
train

label                                              title  \
0         1   Chinese converting to Islam after realizing t...   
1         1   Captions on a reuploaded video about the U.S....   
2         0   Ghana has 307 ambulances with mobile ventilat...   
3         1   “Governor Andy Beshear has authorized Kentuck...   
4         1   Photo shows food being distributed to Rwandan...   
...     ...                                                ...   
4941      1  Canine coronavirus vaccine is for dogs – not r...   
4942      1  Fact Check: Post claiming alcohol kills corona...   
4943      1  Is the ‘Umbrella Corporation’ Logo Oddly Simil...   
4944      1  Images Of Baby With Birth Defect Falsely Linke...   
4945      1  Fact Check: Memes With Advice For Preventing C...   

                                                 author  \
0     https://english.factcrescendo.com/2020/02/20/c...   
1     https://verafiles.org/articles/vera-files-fact...   
2     https://ghanafact.com/fact-check-does-ghanas-3...   
3     https://www.politifact.com/factchecks/2020/may...   
4     https://africacheck.org/fbcheck/food-distribut...   
...                                                 ...   
4941  https://hindi.boomlive.in/fake-news/video-of-a...   
4942  https://www.vishvasnews.com/english/health/fac...   
4943  https://www.snopes.com/fact-check/resident-evi...   
4944  https://www.boomlive.in/fake-news/images-of-ba...   
4945  https://hoax-alert.leadstories.com/3471891-fac...   

                                                   text  
0     The fact behind every news!, Ever since the Wo...  
1     AUTHOR, VERA Files, DATE, May 08, 2020, SHARE,...  
2     Source: Dr Anthony Nsiah Asare, Verdict: Mostl...  
3     More Info, Trying to focus on school work at h...  
4     A photo of hundreds of neat piles of bedding a...  
...                                                 ...  
4941  A photo shared on Facebook in South Africa sho...  
4942  A viral post on social media claims that alcoh...  
4943  , The claim that “Corona” is an anagram for “R...  
4944  BOOM found that the baby was born at a village...  
4945  The most widespread meme is this one, claiming...  

[4720 rows x 4 columns]

## Dealing with duplicate values 

In [29]:
train.drop_duplicates(subset=['label', 'author','title','text'], keep='first')
train

label                                              title  \
0         1   Chinese converting to Islam after realizing t...   
1         1   Captions on a reuploaded video about the U.S....   
2         0   Ghana has 307 ambulances with mobile ventilat...   
3         1   “Governor Andy Beshear has authorized Kentuck...   
4         1   Photo shows food being distributed to Rwandan...   
...     ...                                                ...   
4941      1  Canine coronavirus vaccine is for dogs – not r...   
4942      1  Fact Check: Post claiming alcohol kills corona...   
4943      1  Is the ‘Umbrella Corporation’ Logo Oddly Simil...   
4944      1  Images Of Baby With Birth Defect Falsely Linke...   
4945      1  Fact Check: Memes With Advice For Preventing C...   

                                                 author  \
0     https://english.factcrescendo.com/2020/02/20/c...   
1     https://verafiles.org/articles/vera-files-fact...   
2     https://ghanafact.com/fact-check-does-ghanas-3...   
3     https://www.politifact.com/factchecks/2020/may...   
4     https://africacheck.org/fbcheck/food-distribut...   
...                                                 ...   
4941  https://hindi.boomlive.in/fake-news/video-of-a...   
4942  https://www.vishvasnews.com/english/health/fac...   
4943  https://www.snopes.com/fact-check/resident-evi...   
4944  https://www.boomlive.in/fake-news/images-of-ba...   
4945  https://hoax-alert.leadstories.com/3471891-fac...   

                                                   text  
0     The fact behind every news!, Ever since the Wo...  
1     AUTHOR, VERA Files, DATE, May 08, 2020, SHARE,...  
2     Source: Dr Anthony Nsiah Asare, Verdict: Mostl...  
3     More Info, Trying to focus on school work at h...  
4     A photo of hundreds of neat piles of bedding a...  
...                                                 ...  
4941  A photo shared on Facebook in South Africa sho...  
4942  A viral post on social media claims that alcoh...  
4943  , The claim that “Corona” is an anagram for “R...  
4944  BOOM found that the baby was born at a village...  
4945  The most widespread meme is this one, claiming...  

[4720 rows x 4 columns]

## Save the file 

In [30]:
train.to_csv(op.join(DATA_DIR, 'Covid-19/Claims_to_work_with/claims.csv'), index=False)

In [31]:
from pandas_profiling import ProfileReport

In [32]:
profile = ProfileReport(train, title="Pandas Profiling Report", explorative=True)

In [33]:
profile 

In [34]:
profile.to_file(op.join(DATA_DIR,"Covid-19/Claims_to_work_with/Kaggle_report.html"))

In [35]:
stat=train.author.value_counts()

In [36]:
stat

https://factcheck.afp.com/story-originated-parody-account-no-driver-threatening-spread-covid-19-across-nigeria                   3
https://factcheck.afp.com/health-experts-refute-new-misleading-claims-about-coronavirus-prevention-sri-lanka                     3
https://factcheck.afp.com/graphic-purported-quote-philippine-vice-president-leni-robredo-has-been-doctored                       3
https://www.thequint.com/news/webqoof/china-seeks-approval-to-kill-coronavirus-patients-no-its-a-hoax-website-fact-check         3
https://hoax-alert.leadstories.com/3472004-fact-check-5g-technology-not-believed-to-have-caused-coronavirus.html                 3
                                                                                                                                ..
https://www.rappler.com/newsbreak/fact-check/258115-vico-sotto-quote-vote-next-elections                                         1
https://www.rappler.com/newsbreak/fact-check/261603-brown-sugar-cures-coronavirus  

In [37]:
# Define plot space
fig, ax = plt.subplots(figsize=(30, 9))
# Define x and y axes
ax.bar(stat.index, stat.values, align='center', width=0.50)
# Set plot title and axes labels
ax.set(title = "Number of claims by source")
ax.set_xlabel("author")
ax.set_ylabel("number of claims")
ax.set_xlim(0, 10)
ax.set_ylim(0, 4)
plt.show()